# Managing FABRIC Credentials 

The FABRIC API is used via proxy objects that manage connections to the control framework.<br>

For any experimentation on the FABRIC testbed it is first required take a few steps to establish a connection gateway to the system. This is done via:

1. A credential manager proxy, `Slice Manager`. 
2. Authentication `Tokens`.
3. An `SSH Key`. 

## The Slice Manager 

The `Slice Manager` proxy manages `tokens` used to authorize requests to the system. It is also used to query resource availability, submit requests for a slice, query existing slices/slivers, delete slices and more.

In [ ]:
help(SliceManager)

## Tokens    
Fabric uses two kinds of tokens:

- `Refresh Token`: When a user logins to Jupyterhub after authenticating against CILogon, an initial refresh token is automatically derived. This token is available as the environment variable `CILOGON_REFRESH_TOKEN` and is used to generate the initial Identity Token. On every call to the [refresh_tokens](#refresh) method, Fabric Refresh Token is changed and updated. Each Refresh Token is valid for 24 hours.

- `Identity Token`: The control/measurement framework APIs require an Identity Token. Identity tokens can be generated from a valid Refresh Token at any time. Each Identity Token is valid upto an hour.

- *TODO*:
    - What requests exactly are identity tokens required for? What happens if it is invalid?
    - Should user then keep track of the time and take care to generate a new token every hour?
    - Should user use refresh_tokens to get new refresh token or use the environment variable?

## SSH 
The ssh key is used when requesting to create a slice.
- *TODO*:
    - Why and where is the ssh keys also needed?
    - For how long is the ssh key valid for? Should the user create a new one in every session/multiple times per session or....?

# Step-by-step Setup

## Save the Initial Refresh Token

Retrieve or set the refresh token (exprires 24 hours after login).
- *TODO*:
    - Where is the token stored? localstorage?

In [ ]:
import os

fabric_refresh_token=None
%store -r fabric_refresh_token  #store -r: store OR retrieve

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

## Create Slice Manager

Environment variables `FABRIC_CREDMGR_HOST` and `FABRIC_ORCHESTRATOR_HOST` are passed to the constructor to set the values for the Credential Manager Host and Orchestrator Client Host behind Slice Manager.*

Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters.

- TODO:
    - *Why are the host variables needed?

In [ ]:
import os
import json
from fabrictestbed.slice_manager import SliceManager, Status

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

# Create Slice Manager
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

<a id='refresh'></a>
## Generate/Refresh ID Token

After slice manager has been created, the ID token can be generated or refreshed.

Since ID tokens expire after one hour, the user should rememeber to re-run this command frequently.

Note: the same command, `slice_manager.refresh_tokens()`, is used both to generate the initial ID token *and* to refresh the ID and Refresh tokens.

- *TODO*:
    - Do you need to store the id token??
    - How is it actually used? Only in the background? 

In [1]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("New Id Token: {}".format(fabric_id_token))
print()
print("Stored new Refresh and ID Tokens")
%store fabric_refresh_token 
%store fabric_id_token
print()
print()

Exception occurred while getting tokens:name 'slice_manager' is not defined


NameError: name 'slice_manager' is not defined

## Configure SSH Key

The `ssh_key` is needed as an argument to slice manager.create() when making a new slice, see the [simple demo](/Users/Nanna/Desktop/fabric_nanna/jupyter-examples/fabric/demos/Workshop-Spring2021/simple-workshop-demo.ipynb) ( ***link to notebook on slice creation or simple demo *** )

In [ ]:
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

### (Paramiko SSH for Configuring Nodes)

`??Incude as sub paragraph or not??`